In [1]:
import json
import numpy as np
from pathlib import Path

from data_reader import JsonDocReader, Document

In [9]:
data_reader = JsonDocReader(Path('data/docs.json')).read_documents()
docs = list(data_reader)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sem.kolesnikov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
1236it [13:46,  1.50it/s]


In [20]:
import tqdm
import networkx as nx


sim_matrix = np.zeros((len(docs), len(docs)))


def jaccard_sim(first_doc, second_doc) -> float:
    first_doc_words = set(first_doc.tokens)
    second_doc_words = set(second_doc.tokens)

    return len(first_doc_words & second_doc_words) / len(first_doc_words | second_doc_words)


graph = nx.Graph()
for first_ix, first_doc in tqdm.tqdm(enumerate(docs)):
    for second_ix, second_doc in enumerate(docs[first_ix + 1:]):
        sim = jaccard_sim(first_doc, second_doc)
        sim_matrix[first_ix, second_ix] = sim
        sim_matrix[second_ix, first_ix] = sim
        graph.add_edge(first_doc.name, second_doc.name, weight=sim)

1236it [09:57,  2.07it/s]
